In [7]:
!pip install remotezip 
!pip install tqdm 
!pip install opencv-python==4.5.2.52
!pip install opencv-python-headless==4.5.2.52 
!pip install tf-models-official

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement opencv-python==4.5.2.52 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84)
ERROR: No matching distribution found for opencv-python==4.5.2.52
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement opencv-python-headless==4.5.2.52 (from versions: 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.4

In [1]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

2024-09-05 11:01:15.583157: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-05 11:01:15.727701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 11:01:15.783781: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 11:01:15.798732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-05 11:01:15.899221: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def format_frames(frame, output_size):
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths] 
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

Create the training and test datasets:

In [3]:
from pathlib import Path
batch_size = 8
num_frames = 8

# Define the subset paths based on your downloaded dataset
subset_paths = {
    'train': Path('dataset/train'),
    'test': Path('dataset/test')
}

output_signature = (
    tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'], num_frames, training=True),
    output_signature=output_signature
).batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['test'], num_frames),
    output_signature=output_signature
).batch(batch_size)


I0000 00:00:1725514284.321059   19186 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-05 11:01:24.425584: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
from pathlib import Path

batch_size = 8
num_frames = 8

# Define the subset paths based on your downloaded dataset
subset_paths = {
    'train': Path('dataset/train'),
    'test': Path('dataset/test')
}

output_signature = (
    tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'], num_frames, training=True),
    output_signature=output_signature
).batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['test'], num_frames),
    output_signature=output_signature
).batch(batch_size)

# Collect unique labels from the dataset
unique_labels = set()
for frames, labels in train_ds:
    unique_labels.update(labels.numpy())

# Sort the labels for consistency (optional)
unique_labels = sorted(unique_labels)

# Save the labels to a label.txt file
label_file_path = Path('label.txt')
with label_file_path.open('w') as f:
    for label in unique_labels:
        f.write(f"{label}\n")

print(f"Labels saved to {label_file_path}")


Labels saved to label.txt


In [4]:
for frames, labels in train_ds.take(10):
  print(labels)

tf.Tensor([1 2 1 1 1 2 0 0], shape=(8,), dtype=int16)
tf.Tensor([0 2 2 0 1 2 2 2], shape=(8,), dtype=int16)
tf.Tensor([2 2 2 0 0 1 2 1], shape=(8,), dtype=int16)
tf.Tensor([2 2 0 0 1 0 2 1], shape=(8,), dtype=int16)
tf.Tensor([0 2 0 0 1 2 1 0], shape=(8,), dtype=int16)
tf.Tensor([0 0 0 2 2 1 0 1], shape=(8,), dtype=int16)
tf.Tensor([0 0 1 1 2 1 1 0], shape=(8,), dtype=int16)
tf.Tensor([0 2 2 1 1 1 0 0], shape=(8,), dtype=int16)
tf.Tensor([2 2 1 0 0 1 2 1], shape=(8,), dtype=int16)
tf.Tensor([2 0 1 0 2 1 1 1], shape=(8,), dtype=int16)


2024-09-05 11:02:09.361144: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Take a look at the shape of the data.

In [10]:
import numpy as np
import pathlib

def load_labels():
    """Load the custom labels from the local label.txt file."""
    labels_path = pathlib.Path('label.txt')  # Path to your custom label.txt file
    lines = labels_path.read_text().splitlines()  # Read the file and split it into lines
    return np.array([line.strip() for line in lines])  # Convert lines to a NumPy array



['0' '1' '2']


In [12]:
print(f"Shape: {frames.shape}")
print(f"Label: {labels.shape}")

Shape: (8, 8, 224, 224, 3)
Label: (8,)


In [13]:
gru = layers.GRU(units=4, return_sequences=True, return_state=True)

inputs = tf.random.normal(shape=[1, 10, 8]) # (batch, sequence, channels)

result, state = gru(inputs) # Run it all at once

In [14]:
first_half, state = gru(inputs[:, :5, :])   # run the first half, and capture the state
second_half, _ = gru(inputs[:,5:, :], initial_state=state)  # Use the state to continue where you left off.

print(np.allclose(result[:, :5,:], first_half))
print(np.allclose(result[:, 5:,:], second_half))

True
True


In [15]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model  
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

movinet_a0_base/
movinet_a0_base/checkpoint
movinet_a0_base/ckpt-1.data-00000-of-00001
movinet_a0_base/ckpt-1.index


In [16]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [17]:
model = build_classifier(batch_size, num_frames, resolution, backbone, 10)

In [18]:
num_epochs = 5
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(loss=loss_obj, optimizer='adam', metrics=['accuracy'])


In [19]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=num_epochs,
                    validation_freq=1,
                    verbose=1)

Epoch 1/5


I0000 00:00:1725449354.024196   65996 service.cc:146] XLA service 0x779fb0029c60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725449354.024220   65996 service.cc:154]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1725449354.033781   65996 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


      1/Unknown - 12s 12s/step - loss: 2.3192 - accuracy: 0.0000e+00

2024-09-04 16:59:20.835865: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


38/38 [==============================] - 313s 8s/step - loss: 0.5215 - accuracy: 0.8800 - val_loss: 0.0457 - val_accuracy: 1.0000
Epoch 2/5


2024-09-04 17:04:21.681861: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


38/38 [==============================] - 302s 8s/step - loss: 0.0662 - accuracy: 0.9767 - val_loss: 0.0340 - val_accuracy: 1.0000
Epoch 3/5
38/38 [==============================] - 300s 8s/step - loss: 0.0295 - accuracy: 0.9967 - val_loss: 0.0195 - val_accuracy: 1.0000
Epoch 4/5
38/38 [==============================] - 298s 8s/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 0.0168 - val_accuracy: 1.0000
Epoch 5/5
38/38 [==============================] - 298s 8s/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.0169 - val_accuracy: 1.0000


In [25]:
model.evaluate(test_ds, return_dict=True)


11/11 [==============================] - 67s 6s/step - loss: 0.0195 - accuracy: 1.0000


2024-09-04 17:30:09.366744: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


{'loss': 0.01945176161825657, 'accuracy': 1.0}

In [26]:
model.save("Movinet_Model.h5")
print("Model saved successfully...")

Model saved successfully...


/home/ai/.local/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:

def test_model_with_webcam(model_path):
    def format_frames(frame, output_size):
        frame = tf.image.convert_image_dtype(frame, tf.float32)
        frame = tf.image.resize_with_pad(frame, *output_size)
        return frame

    def get_custom_objects():
        return {
            'MovinetClassifier': movinet_model.MovinetClassifier
        }

    num_frames = 8
    output_size = (224, 224)
    cap = cv2.VideoCapture(1)

    # Load the saved model with custom objects
    custom_objects = get_custom_objects()
    model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    frame_buffer = []

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Error: Failed to capture frame.")
            break

        # Preprocess the frame
        processed_frame = format_frames(frame, output_size)
        frame_buffer.append(processed_frame)

        if len(frame_buffer) == num_frames:
            # Stack frames into a batch
            input_frames = np.expand_dims(frame_buffer, axis=0)

            # Make predictions
            predictions = model.predict(input_frames)
            predicted_class = np.argmax(predictions[0])

            # Display the prediction on the frame
            label = f"Predicted: {predicted_class}"
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Clear the frame buffer
            frame_buffer = []

        # Show the frame with predictions
        cv2.imshow("Webcam", frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


test_model_with_webcam("Movinet_Model.h5")

[ WARN:0@2992.657] VIDEOIO(V4L2:/dev/video1): can't open camera by index


ValueError: Unknown config_item: 'Vision>Movinet'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.